In [1]:
import pandas as pd, numpy as np, re
pd.set_option("display.max_columns", 120)

raw_path = "../data/raw/listings.csv.gz"
df = pd.read_csv(raw_path, low_memory=False)
len(df), df.columns[:12]


(14187,
 Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
        'description', 'neighborhood_overview', 'picture_url', 'host_id',
        'host_url', 'host_name'],
       dtype='object'))

In [2]:
df["price_eur"] = (df["price"].astype(str)
                   .str.replace(r"[€$,]", "", regex=True)
                   .str.replace(r"\s", "", regex=True)
                   .astype(float))
df["price_eur"].describe()


count     9183.000000
mean       211.728302
std       1653.597912
min          5.000000
25%         70.000000
50%        108.000000
75%        168.000000
max      50000.000000
Name: price_eur, dtype: float64

In [3]:
if "bathrooms_text" in df.columns:
    df["bathrooms_num"] = (df["bathrooms_text"].astype(str)
                           .str.extract(r"(\d+(\.\d+)?)")[0]
                           .astype(float))


In [4]:
keep = ["id","latitude","longitude","neighbourhood_cleansed","room_type","property_type",
        "accommodates","bedrooms","bathrooms_num","minimum_nights","price_eur",
        "review_scores_rating","number_of_reviews"]
df = df[[c for c in keep if c in df.columns]].copy()
df.head(3)


,id,latitude,longitude,neighbourhood_cleansed,room_type,property_type,accommodates,bedrooms,bathrooms_num,minimum_nights,price_eur,review_scores_rating,number_of_reviews
0,3176,52.53471,13.41810,Prenzlauer Berg Südwest,Entire home/apt,Entire rental unit,2,1.0,1.0,63,105.0,4.63,149
1,9991,52.53269,13.41805,Prenzlauer Berg Südwest,Entire home/apt,Entire rental unit,7,4.0,2.5,6,135.0,5.00,7
2,14325,52.54813,13.40366,Prenzlauer Berg Nordwest,Entire home/apt,Entire rental unit,1,0.0,1.0,150,75.0,4.68,26


In [5]:
# drop rows missing essentials
df = df.dropna(subset=["price_eur","latitude","longitude"]).drop_duplicates(subset=["id"])

# cap extreme prices to 1st–99th percentile
q1, q99 = df["price_eur"].quantile([0.01, 0.99])
df["price_eur"] = df["price_eur"].clip(q1, q99)

# light NA imputation for numerics
for col in ["bedrooms","bathrooms_num","review_scores_rating","number_of_reviews","minimum_nights","accommodates"]:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

df.describe().T.head(12)


,count,mean,std,min,25%,50%,75%,max
id,9183.0,6.347248e+17,5.610167e+17,3176.000000,3.300486e+07,7.706750e+17,1.162154e+18,1.447175e+18
latitude,9183.0,5.250866e+01,3.549185e-02,52.340027,5.249111e+01,5.250968e+01,5.253158e+01,5.265076e+01
longitude,9183.0,1.340046e+01,7.363779e-02,13.118150,1.335607e+01,1.340628e+01,1.343717e+01,1.373727e+01
accommodates,9183.0,3.312316e+00,2.119453e+00,1.000000,2.000000e+00,2.000000e+00,4.000000e+00,1.600000e+01
bedrooms,9183.0,1.329413e+00,8.508024e-01,0.000000,1.000000e+00,1.000000e+00,2.000000e+00,1.400000e+01
bathrooms_num,9183.0,1.158445e+00,4.800488e-01,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01
minimum_nights,9183.0,2.673342e+01,4.433157e+01,1.000000,2.000000e+00,3.000000e+00,3.000000e+01,9.990000e+02
price_eur,9183.0,1.365479e+02,1.012566e+02,28.000000,7.000000e+01,1.080000e+02,1.680000e+02,6.595400e+02
review_scores_rating,9183.0,4.768335e+00,3.456945e-01,0.000000,4.730000e+00,4.850000e+00,4.930000e+00,5.000000e+00
number_of_reviews,9183.0,5.772634e+01,1.144863e+02,0.000000,1.000000e+00,1.300000e+01,6.300000e+01,2.877000e+03


In [6]:
out_path = "../data/processed/listings_clean.csv"
df.to_csv(out_path, index=False)
out_path, df.shape


('../data/processed/listings_clean.csv', (9183, 13))

- Preis als float geparst, Ausreißer auf 1.–99. Perzentil gecappt.
- Wichtige Spalten ausgewählt; fehlende numerische Werte mit Median gefüllt.
- Duplikate pro Listing-ID entfernt.
